In [25]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize
from qiskit.algorithms.optimizers import SPSA

In [12]:
def control_b(auxiliary, qubits):

    for ia in qubits:
        circ.ch(auxiliary, ia)

        
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg):

    circ.h(auxiliary_index)

    control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(auxiliary_index, qubits[ty])


    control_b(auxiliary_index, qubits)
    
    circ.h(auxiliary_index)        

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)

    for ie in range (0, len(gate_type[0])):
        if (gate_type[0][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])

    for ie in range (0, len(gate_type[1])):
        if (gate_type[1][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)


def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[2])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

        
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[1], 4)
    circ.cz(qubits[0], 4)
    circ.ccx(auxiliary, qubits[1], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[2], 4)
    circ.cz(qubits[1], 4)
    circ.ccx(auxiliary, qubits[2], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

In [18]:
#A noisy impelemntation of the code. It samples the circuit many times which results in noise (Found in the qiskit tutorial)

def calculate_cost_function_noise(parameters):

    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9]]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(1)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1, 2, 3], 0, parameters)

            circ.measure(0, 0)

            t_circ = transpile(circ, backend)
            qobj = assemble(t_circ, shots=10000)
            job = backend.run(qobj)

            result = job.result()
            outputstate = result.get_counts(circ)

            if ('1' in outputstate.keys()):
                m_sum = float(outputstate["1"])/100000
            else:
                m_sum = 0

            overall_sum_1+=multiply*(1-2*m_sum)

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(1)
                
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.measure(0, 0)

                t_circ = transpile(circ, backend)
                qobj = assemble(t_circ, shots=10000)
                job = backend.run(qobj)

                result = job.result()
                outputstate = result.get_counts(circ)

                if ('1' in outputstate.keys()):
                    m_sum = float(outputstate["1"])/100000
                else:
                    m_sum = 0

                mult = mult*(1-2*m_sum)
            
            overall_sum_2+=multiply*mult
            
    print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [16]:
def calc_results(output, coefficient_set):
    
    out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

    circ = QuantumCircuit(3, 3)
    apply_fixed_ansatz([0, 1, 2], out_f)
    circ.save_statevector()

    backend = Aer.get_backend('aer_simulator')
    t_circ = transpile(circ, backend)
    qobj = assemble(t_circ)
    job = backend.run(qobj)

    result = job.result()
    o = result.get_statevector(circ, decimals=10)

    a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
    a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
    a3 = np.add(a1, a2)

    b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

    return (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
    

In [23]:
coefficient_set = [0.55, 0.45]
gate_set = [[0, 0, 0], [0, 0, 1]]

out = minimize(calculate_cost_function_noise, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

minNoise = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(minNoise)

0.13222906647795307
0.15920584151668016
0.0968101466911061
0.1022718060953256
0.13520087815274928
0.07873569388557577
0.08170903553850828
0.11951027133397163
0.07476688862025138
0.060128140409413056
0.06780128870213287
0.07841886238440532
0.058572957135948234
0.06121174711277999
0.05161503841012416
0.05024496057026795
0.05485712290209832
0.04674525326078849
0.056857105243362516
0.04821331731634748
0.04401249146092434
0.043535156172489264
0.04051979451007259
0.04288720065353824
0.04231742903664759
0.04168053167111996
0.044030223595671214
0.04307072758175934
0.041578989948152456
0.041850186859449856
0.04231561937959527
0.044040854595383094
0.04080222530159017
0.0415460375221266
0.04092367803157537
0.04153473331997537
0.04079017085485048
0.04216514527184301
0.040252803517276114
0.040675180591266735
0.041571271640268526
0.04164374835655926
0.040836267590796504
0.04065932121759963
0.04116149201671082
0.04237068476322714
0.041689276094872385
0.04117518878111315
0.0411917167333552
0.041639666

In [21]:
print(minNoise)
print(out)

(0.12499999999999997+0j)
     fun: 0.051673994313553684
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 89
  status: 1
 success: True
       x: array([2.4877346 , 1.7509771 , 1.24592234, 1.61290517, 3.81613983,
       2.81129193, 0.53684332, 3.52764195, 2.42480361])


In [14]:
# Implements the entire cost function on the quantum circuit

def calculate_cost_function_clean(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9]]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(5)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1, 2, 3], 0, parameters)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            qobj = assemble(t_circ)
            job = backend.run(qobj)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(5)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.save_statevector()    
                t_circ = transpile(circ, backend)
                qobj = assemble(t_circ)
                job = backend.run(qobj)

                result = job.result()
                outputstate = np.real(result.get_statevector(circ, decimals=100))
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = o[l]**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult
            
    print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [24]:
coefficient_set = [0.55, 0.45]
gate_set = [[0, 0, 0], [0, 0, 1]]

out = minimize(calculate_cost_function_clean, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

minClean = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(minClean)

0.9986403116455076
0.9407543088863212
0.9203644609272046
0.9714651532910953
0.9213635993374634
0.8289715075198553
0.9300355530102139
0.6978070240986827
0.8804957875847726
0.6392952111262176
0.5800020144802147
0.5649595702340056
0.571779445140892
0.8375873060496155
0.5956012042236691
0.5290590155995516
0.4991623738787122
0.516064290390897
0.5392741135424075
0.5011809328881089
0.48190679843261597
0.6664099821833201
0.5008217709100238
0.5084025380242456
0.48210522732513583
0.491854078811204
0.4971424626851658
0.5031817566978418
0.5297893678364981
0.4785101968257741
0.4781913291808505
0.46609040733938956
0.48434114107428505
0.47443117471325613
0.4570125421042168
0.4342323817154474
0.43578791037091735
0.42830140554230456
0.42847742360785357
0.42957846145391565
0.44179519786883437
0.4101971651418851
0.4394410598046916
0.40368456765372296
0.39168264856672497
0.3719722410279458
0.36058333885989136
0.406796197835377
0.4054764126461199
0.24586298718023003
0.23375839432248202
0.3069851823501144
0

In [28]:
spsa = SPSA(maxiter=300)

coefficient_set = [0.55, 0.45]
gate_set = [[0, 0, 0], [0, 0, 1]]

#results = spsa.optimize(9, calculate_cost_function_noise, initial_point=[float(random.randint(0,3000))/1000 for i in range(0, 9)])

print(results)
out_f = [results[0][0:3], results[0][3:6], results[0][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

spsaNoise = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(spsaNoise)


results = spsa.optimize(9, calculate_cost_function_clean, initial_point=[float(random.randint(0,3000))/1000 for i in range(0, 9)])

print(results)
out_f = [results[0][0:3], results[0][3:6], results[0][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

spsaClean = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(spsaClean)


(array([ 4.78022707,  1.85103339,  1.17718023, -0.46117518,  0.70903902,
        1.15851433,  1.05403743,  4.08303708,  3.14995235]), 0.04498469236826541, 600)
(0.5631307801100848+0j)
0.966752411798589
0.8311861508103381
0.8921365208951622
0.9886677149285664
0.7643204401820205
0.9481200551250312
0.9565942299494465
0.957287537900471
0.9910720188948259
0.9998821174161457
0.9897389415778511
0.9998360901492721
0.9999556474204387
0.995944369011738
0.9413849976364658
0.8463207828780278
0.9221947010399295
0.8056381726974684
0.9815167200860712
0.9716709737431022
0.9905352659473204
0.9933776926435415
0.9892527386603612
0.9958574877017609
0.9371265653891445
0.9919248181971216
0.9970089671676323
0.9944224395107651
0.9973938055858882
0.9111939111253756
0.9434172942306143
0.9145874783689953
0.9999996636743725
0.955328540382173
0.9492481098634304
0.9454830344647115
0.9374107541791863
0.9910722589843987
0.9972492456414962
0.9984668277801777
0.9745481449533998
0.9838360737126479
0.9980199033512367
0.9

0.28429770095700824
0.3613021738857959
0.4328886167345628
0.32505206076801196
0.37090514224310733
0.3126394855961129
0.3881917235481721
0.3610409196520421
0.3678387411603484
0.3399585795740153
0.27457528397964703
0.41233164444407455
0.27387350469592653
0.316416115408531
0.3047911041709217
0.49631701432508757
0.4336889460911795
0.40318200210968436
0.33673888700906984
0.40050907096083954
0.32490007765459794
0.33433368124577834
0.33794986059594734
0.2824443976030855
0.548572124608625
0.5397922748179629
0.479755917189547
0.5445958057466642
0.378677335716161
0.5725708660217537
0.5900111753075341
0.27978208718944486
0.48389753681935344
0.254645284794927
0.6015941260293058
0.5617079948404922
0.9900359512584206
0.6462761064412843
0.4958841948119802
0.371475761968055
0.5837398643932012
0.32327489804409903
0.3563257610815994
0.4241856084585047
0.24456824184093306
0.240132480246111
0.37995937833694804
0.21406901995047112
0.326122221032635
0.2949857627384389
0.3759257455367686
0.3921670557727346
0

# Coefficents with len 3

In [29]:
### Having a coefficent set of len 3
coefficient_set = [0.55, 0.225, 0.225]
gate_set = [[0, 0, 0], [0, 1, 0], [0, 0, 1]]

out = minimize(calculate_cost_function_noise, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[2]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a0 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,-1,0,0,0,0,0], [0,0,0,-1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])

a3 = np.add(np.add(a2, a0), a1)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

minNoise = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(minNoise)

0.24362166135989016
0.26915637069223086
0.2118981863696019
0.21065257206632337
0.18712998774046152
0.21471567666958213
0.1942797729075818
0.1530149502479693
0.15757852143167295
0.14392450405385082
0.08665951270449124
0.07787534547946129
0.07255903123669272
0.08089480856844011
0.06532851768892611
0.05616528874650428
0.04965783137665725
0.05872358158328328
0.046528915352633815
0.05347437370041874
0.04491229372579464
0.04462299885200005
0.0528742402637975
0.047002524601482576
0.03829266518538521
0.03871182493424841
0.039007741664071505
0.054479857738696014
0.05435355282793186
0.03938765893980023
0.04003747798811741
0.03916845920831835
0.030356138342221506
0.03493826236040076
0.03101724444170484
0.03463545466504314
0.032398143131809554
0.03292689226798029
0.03240394888054421
0.030325071300911555
0.029936438784553987
0.029679247185309343
0.031896970997699614
0.029296563120918773
0.029832592663150304
0.029504648012451007
0.029343734758963236
0.029724579456878453
0.029410392154662235
0.030804

In [30]:
### Having a coefficent set of len 3
coefficient_set = [0.55, 0.225, 0.225]
gate_set = [[0, 0, 0], [0, 1, 0], [0, 0, 1]]

out = minimize(calculate_cost_function_clean, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[2]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a0 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,-1,0,0,0,0,0], [0,0,0,-1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])

a3 = np.add(np.add(a2, a0), a1)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

minClean = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(minClean)

0.9524223849666272
0.9296468098768509
0.9477240820446668
0.9778182657837763
0.9438230965423977
0.9166344091838765
0.9960399671078983
0.7691036384252781
0.7351098330650641
0.7384510432307058
0.598939549317006
0.7559862786832232
0.6560563045916055
0.6869789998995102
0.6753681411898079
0.6593965175814205
0.7176158550645703
0.41736061362747956
0.6706589372809173
0.43971852467653494
0.6049292108157283
0.48804549409558373
0.4016498563950046
0.4243043470607174
0.42862626093388634
0.35291983790081505
0.31175058951934653
0.40597122180138023
0.3379995020066393
0.24335434004936662
0.19805654270313588
0.30273715564203085
0.2619215987585426
0.25945704782856016
0.18000306187501347
0.3879572567177032
0.17028668710854467
0.3422515054718731
0.22641749938119693
0.16516192110690286
0.17283420317524267
0.18245083695373465
0.19295633448232208
0.166214042436872
0.19614491231120423
0.15715006620464556
0.16862860500353694
0.15815599976381978
0.175080953955151
0.14986700802247632
0.13813860740455408
0.13186545

In [31]:
### Having a coefficent set of len 3
coefficient_set = [0.55, 0.225, 0.225]
gate_set = [[0, 0, 0], [0, 1, 0], [0, 0, 1]]

spsa = SPSA(maxiter=300)

out = spsa.optimize(9, calculate_cost_function_noise, initial_point=[float(random.randint(0,3000))/1000 for i in range(0, 9)])
print(out)

out_f = [out[0][0:3], out[0][3:6], out[0][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[2]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a0 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,-1,0,0,0,0,0], [0,0,0,-1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])

a3 = np.add(np.add(a2, a0), a1)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

spsaNoise = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(spsaNoise)

0.06416812362646751
0.09093314624860138
0.07335923506595865
0.0844950060828592
0.08544321525989695
0.08630862797377714
0.09055084398060598
0.06940452007897058
0.07896995127594808
0.09033432370677064
0.08684323326182142
0.08351870661033844
0.07858916480169154
0.07378144216913896
0.08394061108905371
0.08532841059462981
0.08655654479597186
0.07401102508040847
0.08176142370945161
0.07662280828458834
0.07299557049819516
0.08676632068746626
0.07572145840459055
0.08873454853544205
0.08415330935887189
0.08418355194968785
0.09147550447129826
0.06624638007556172
0.0827095445348579
0.0832442488680426
0.08047591968089007
0.08540146435796003
0.07856284466008978
0.07967155537462867
0.09985205981954692
0.0736752836840383
0.07451346188243635
0.08459545249037159
0.07824510279318275
0.09263015263476326
0.07769783073654413
0.08620284515397825
0.08437668807283871
0.08029751145937714
0.07600297367990472
0.08012597358262463
0.07894347494321385
0.0825285945266856
0.09921175900105672
0.07347338920953872
0.075

0.03161064246902001
0.0327119151274019
0.03247990620471275
0.029206850559054365
0.028908182501553026
0.029108251734723867
0.029218969272279405
0.029985136373937182
0.030024391364140368
0.02898239518712209
0.029182482649793173
0.032712399630024924
0.030871546500171654
0.03016216156960072
0.029825656392126865
0.031468646432352365
0.031459044254539714
0.031038585775555183
0.030652844465742124
0.030798800629176948
0.031019226411742484
0.031030633841037325
0.0317854740694119
0.03141571213740291
0.031245559730749828
0.029885226885594585
0.02980737577958814
0.03236223747491451
0.03222966549714912
0.031087088216578462
0.030321642609616672
0.03559829796881775
0.03629261112289428
0.029125551423712315
0.029033683745814054
0.03200234493648546
0.031895818338014026
0.029773471259483375
0.029510620285730704
0.02856859725327776
0.02977799055751329
0.03045615531491941
0.03050966279394851
0.028868339784787644
0.029520521156310542
0.029107201925885562
0.029221450422915685
0.029760543993749833
0.030151259

In [32]:
### Having a coefficent set of len 3
coefficient_set = [0.55, 0.225, 0.225]
gate_set = [[0, 0, 0], [0, 1, 0], [0, 0, 1]]

spsa = SPSA(maxiter=300)

out = spsa.optimize(9, calculate_cost_function_clean, initial_point=[float(random.randint(0,3000))/1000 for i in range(0, 9)])
print(out)

out_f = [out[0][0:3], out[0][3:6], out[0][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[2]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a0 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,-1,0,0,0,0,0], [0,0,0,-1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])

a3 = np.add(np.add(a2, a0), a1)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

spsaClean = (b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2
print(spsaClean)

0.3622593392688206
0.3860001568769431
0.38007825250996696
0.48101349671022386
0.4431273705090061
0.38215268822637904
0.4221119441374894
0.3991197681550349
0.3349280042641788
0.3846746978891519
0.3153867687384265
0.39030491942232826
0.4352250206697149
0.3721312855961245
0.5276369812767057
0.248761530734534
0.38602030678313815
0.37563856652837135
0.23690614708783964
0.573879394693105
0.3322581368850819
0.3851759036134993
0.3180541560915411
0.37395910773154484
0.40342142175187623
0.40097440282199426
0.40554382511527065
0.3011913925234445
0.248761530734534
0.5276369812767057
0.41688507926393625
0.33721521849942326
0.34587829794510183
0.47821176211654104
0.4501653541404733
0.26993319506410096
0.3959928053972074
0.4591363257904709
0.49248944768955927
0.38210979057016214
0.3343083782611521
0.40683743365756986
0.4639618586913695
0.39650900013778434
0.48486250101267747
0.36749405724132855
0.3734294295857793
0.4166330184617236
0.4650432179245936
0.33651147063772013
0.34342084145316365
0.38577253

0.07529359686439074
0.41018373133513386
0.26898799693604614
0.25576908629324235
0.21731729138339884
0.29512042061803856
0.11360747433389629
0.27412896958646193
0.14148535663452388
0.1150457667854684
0.049443386763431296
0.28398707168868986
0.19243508789834152
0.0648829532424795
0.12989517523940708
0.22303338503250325
0.15648337161869308
0.18150651761287095
0.06858259520877708
0.15512625450793194
0.3229998421917142
0.2342815668361219
0.18111281550746794
0.2035242259071801
0.1899151001822338
0.2520175332588296
0.311592374318439
0.15519209399681466
0.3616149391489163
0.2945577940624967
0.2336620572882191
0.19427059590326068
0.5527453568872587
0.19938225160988698
0.41127268380698523
0.21360335209265047
0.21303083336033213
0.13717163073593497
0.3801640349467351
0.1816705562372476
0.2198184368391033
0.12491036233013386
0.1965014235912621
0.2582182491099596
0.26345834367292886
0.1878701124269958
0.10106791559660389
0.16547206078005794
0.142049415847737
0.17537901962640323
0.14292796960146503
